# 12장. 모델 선택

이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/machine-learning-with-python-cookbook/blob/master/12.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/machine-learning-with-python-cookbook/blob/master/12.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

이 장의 코드는 많은 경고를 발생시킵니다. 이 경고는 대부분 향후 기능 변화를 위한 안내입니다. 출력 결과를 보기 편하도록 경고를 무시하겠습니다. 발생되는 경고는 다음과 같습니다.

- (FutureWarning) GridSearchCV의 cv 매개변수 기본값이 0.22 버전에서 3에서 5로 바뀝니다.
- (FutureWarning) LogisticRegression의 solver 매개변수의 기본값이 0.22 버전에서 liblinear에서 lbfgs로 바뀝니다. multi_class의 기본값은 0.22 버전에서 auto로 바뀝니다.
- (ConvergenceWarning) LogisticRegression이 max_iter에서 지정한 반복 횟수 안에 수렴하지 않으면 경고가 발생합니다. 이 매개변수의 기본값은 100입니다.

In [1]:
import warnings
warnings.filterwarnings("ignore")

## 12.1 완전 탐색을 사용해 최선의 모델 선택하기

In [2]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 로지스틱 회귀 모델을 만듭니다.
logistic = linear_model.LogisticRegression()

# 페널티(penalty) 하이퍼파라미터 값의 후보를 만듭니다.
penalty = ['l1', 'l2']

# 규제 하이퍼파라미터 값의 후보 범위를 만듭니다.
C = np.logspace(0, 4, 10)

# 하이퍼파라미터 후보 딕셔너리를 만듭니다.
hyperparameters = dict(C=C, penalty=penalty)

# 그리드 서치 객체를 만듭니다.
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

# 그리드 서치를 수행합니다.
best_model = gridsearch.fit(features, target)

In [3]:
np.logspace(0, 4, 10)

array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04])

In [4]:
# 최선의 하이퍼파라미터를 확인합니다.
print('가장 좋은 페널티:', best_model.best_estimator_.get_params()['penalty'])
print('가장 좋은 C 값:', best_model.best_estimator_.get_params()['C'])

가장 좋은 페널티: l1
가장 좋은 C 값: 7.742636826811269


In [5]:
# 타깃 벡터를 예측합니다.
best_model.predict(features)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## 12.2 랜덤 서치를 사용하여 최선의 모델 선택하기

In [6]:
# 라이브러리를 임포트합니다.
from scipy.stats import uniform
from sklearn import linear_model, datasets
from sklearn.model_selection import RandomizedSearchCV

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 로지스틱 회귀 모델을 만듭니다.
logistic = linear_model.LogisticRegression()

# 페널티 하이퍼파라미터 후보를 만듭니다. penalty hyperparameter values
penalty = ['l1', 'l2']

# 규제 하이퍼파라미터 값의 후보를 위한 분포를 만듭니다.
C = uniform(loc=0, scale=4)

# 하이퍼파라미터 옵션을 만듭니다.
hyperparameters = dict(C=C, penalty=penalty)

# 랜덤 서치 객체를 만듭니다.
randomizedsearch = RandomizedSearchCV(
    logistic, hyperparameters, random_state=1, n_iter=100, cv=5, verbose=0,
    n_jobs=-1)

# 랜덤 서치를 수행합니다.
best_model = randomizedsearch.fit(features, target)

In [7]:
# 0~4 사이의 균등 분포를 정의하고 10개의 값을 샘플링합니다.
uniform(loc=0, scale=4).rvs(10)

array([0.98583588, 2.93425661, 0.64234633, 2.83296747, 2.04226139,
       1.82818539, 3.67087227, 1.37250589, 0.2745786 , 0.94763201])

In [8]:
# 최선의 하이퍼파라미터를 확인합니다.
print('가장 좋은 페널티:', best_model.best_estimator_.get_params()['penalty'])
print('가장 좋은 C 값:', best_model.best_estimator_.get_params()['C'])

가장 좋은 페널티: l1
가장 좋은 C 값: 1.668088018810296


In [9]:
# 타깃 벡터를 예측합니다.
best_model.predict(features)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## 12.3 여러 학습 알고리즘에서 최선의 모델 선택하기

In [10]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# 랜덤 시드를 설정합니다.
np.random.seed(0)

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 파이프라인을 만듭니다.
pipe = Pipeline([("classifier", RandomForestClassifier())])

# 후보 학습 알고리즘과 하이퍼파라미터로 딕셔너리를 만듭니다.
search_space = [{"classifier": [LogisticRegression()],
                 "classifier__penalty": ['l1', 'l2'],
                 "classifier__C": np.logspace(0, 4, 10)},
                {"classifier": [RandomForestClassifier()],
                 "classifier__n_estimators": [10, 100, 1000],
                 "classifier__max_features": [1, 2, 3]}]

# 그리드 서치 객체를 만듭니다.
gridsearch = GridSearchCV(pipe, search_space, cv=5, verbose=0)

# 그리드 서치를 수행합니다.
best_model = gridsearch.fit(features, target)

In [11]:
# 최선의 모델을 확인합니다.
best_model.best_estimator_.get_params()["classifier"]

LogisticRegression(C=7.742636826811269, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
# 타깃 벡터를 예측합니다.
best_model.predict(features)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## 12.4 전처리와 함께 최선의 모델 선택하기

In [13]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 랜덤 시드를 설정합니다.
np.random.seed(0)

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# StandardScaler와 PCA를 포함한 전처리 객체를 만듭니다.
preprocess = FeatureUnion([("std", StandardScaler()), ("pca", PCA())])

# 파이프라인을 만듭니다.
pipe = Pipeline([("preprocess", preprocess),
                 ("classifier", LogisticRegression())])

# 후보 값을 정의합니다.
search_space = [{"preprocess__pca__n_components": [1, 2, 3],
                 "classifier__penalty": ["l1", "l2"],
                 "classifier__C": np.logspace(0, 4, 10)}]

# 그리드 서치 객체를 만듭니다.
clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)

# 그리드 서치를 수행합니다.
best_model = clf.fit(features, target)

In [14]:
# 최선의 주성분 개수를 확인합니다.
best_model.best_estimator_.get_params()['preprocess__pca__n_components']

1

### 붙임

In [15]:
clf.best_score_

0.96

In [16]:
clf.best_estimator_.named_steps["preprocess"].transform(features[0:1])

array([[-0.90068117,  1.01900435, -1.34022653, -1.3154443 , -2.68412563]])

In [17]:
pipe = Pipeline([("std", StandardScaler()),
                 ("pca", PCA()),
                 ("classifier", LogisticRegression())],
                 memory='cache')

# 후보 값을 정의합니다.
search_space = [{"pca__n_components": [1, 2, 3],
                 "classifier__penalty": ["l1", "l2"],
                 "classifier__C": np.logspace(0, 4, 10)}]

# 그리드 서치 객체를 만듭니다.
clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)

# 그리드 서치를 수행합니다.
best_model = clf.fit(features, target)

In [18]:
clf.best_score_

0.9666666666666667

In [19]:
# 최선의 주성분 개수를 확인합니다.
clf.best_estimator_.get_params()['pca__n_components']

3

In [20]:
clf.best_estimator_.named_steps["pca"].transform(features[0:1])

array([[ 2.64026976,  5.2040413 , -2.48862071]])

## 12.5 병렬화로 모델 선택 속도 높이기

In [21]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 로지스틱 회귀 모델을 만듭니다.
logistic = linear_model.LogisticRegression()

# 규제 페널티의 후보를 만듭니다.
penalty = ["l1", "l2"]

# C 값의 후보 범위를 만듭니다.
C = np.logspace(0, 4, 1000)

# 하이퍼파라미터 옵션을 만듭니다.
hyperparameters = dict(C=C, penalty=penalty)

# 그리드 서치 객체를 만듭니다.
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=-1, verbose=1)

# 그리드 서치를 수행합니다.
best_model = gridsearch.fit(features, target)

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 3236 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:   14.6s finished


In [22]:
# 하나의 코어만 사용하는 그리드 서치 객체를 만듭니다.
clf = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=1, verbose=1)

# 그리드 서치를 수행합니다.
best_model = clf.fit(features, target)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:   53.6s finished


## 12.6 알고리즘에 특화된 기법을 사용하여 모델 선택 수행 속도 높이기

In [23]:
# 라이브러리를 임포트합니다.
from sklearn import linear_model, datasets

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 교차검증 로지스틱 회귀 모델을 만듭니다.
logit = linear_model.LogisticRegressionCV(Cs=100)

# 모델을 훈련합니다.
logit.fit(features, target)

LogisticRegressionCV(Cs=100, class_weight=None, cv='warn', dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

## 12.7 모델 선택 후 성능 평가

In [24]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV, cross_val_score

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 로지스틱 회귀 모델을 만듭니다.
logistic = linear_model.LogisticRegression(solver='liblinear', multi_class='auto')

# Create range of 20 candidate values for C
C = np.logspace(0, 4, 20)

# 하이퍼파라미터 옵션을 만듭니다.
hyperparameters = dict(C=C)

# 그리드 서치 객체를 만듭니다.
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=-1, verbose=0, iid=False)

# 중첩 교차검증을 수행하고 평균 점수를 출력합니다.
cross_val_score(gridsearch, features, target, cv=3).mean()

0.9534313725490197

In [25]:
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=1, iid=False)

In [26]:
best_model = gridsearch.fit(features, target)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [27]:
scores = cross_val_score(gridsearch, features, target, cv=3)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished
